# CDCS Data Management

This Notebook details the functions and interactions for managing data in a CDCS instance.  Primarily, this means having an account on the database allowing you to log in to create and modify records. 

Notes on CDCS design:

- Each record is assigned to a user and can be assigned to a workspace.
- Any records without a workspace are only accessible to the assigned user.
- A workspace serves as a group of records that can be made accessible to specific users.
- Each CDCS instance has a "Global Public Workspace" that is meant for all users to be able to see.

In [1]:
from pathlib import Path

import cdcs
from cdcs import CDCS

print('Notebook executed for cdcs version', cdcs.__version__)

Notebook executed for cdcs version 0.1.5


## 1. Class initialization

A CDCS client manager can be initialized by passing it the host url and authenication information.

Parameters

- __host__: (*str*) URL for the database's server.
- __username__: (*str, optional*) Username of desired account on the server. A prompt will ask for the username if not given.
- __password__: (*str, optional*) Password of desired account on the server.  A prompt will ask for the password if not given.
- __auth__: (*tuple, optional*) Auth tuple to enable Basic/Digest/Custom HTTP Auth.  Alternative to giving username and password seperately.
- __cert__: (*str, optional*) if String, path to ssl client cert file (.pem). If Tuple, (‘cert’, ‘key’) pair.
- __certification__: (*str, optional*) Alias for cert. Retained for compatibility.
- __verify__: (*bool or str, optional*) Either a boolean, in which case it controls whether we verify the server’s TLS certificate, or a string, in which case it must be a path to a CA bundle to use. Defaults to True.

In [2]:
curator = CDCS('https://potentials.nist.gov/', username='lmh1')

Enter password for lmh1 @ https://potentials.nist.gov: ················


## 2. Query data

The query() method will return *all* matching records that you have access to.

Parameters

- __template__: (*list, str, pandas.Series or pandas.DataFrame, optional*) One or more templates or template titles to limit the search by.
- __title__: (*str, optional*) Record title to limit the search by.
- __keyword__: (*str or list, optional*) Keyword(s) to use for a string-based search of record content.  Only records containing all keywords will be returned. 
- __mongoquery__: (*str or dict, optional*) Mongodb find query to use in limiting searches by record element fields.  Note: only record parsing is supported, not field projection.
- __page__: (*int or None, optional*) If an int, then will return results only for that page of 10 records.  If None (default), then results for all pages will be compiled and returned.

Returns
- (*pandas.DataFrame*) All records matching the search request

Specify a template in the database to interact with

In [3]:
# Note: template should the name of a template in the database you are accessing!
template = 'FAQ'

Use query to fetch records

In [4]:
records = curator.query(template=template)
records

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date,template_title
0,5fb55fac26ed1e003a1712ef,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,formats,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:48.764000Z,2020-11-18T17:53:48.764000Z,2020-11-19T13:44:31.936000Z,FAQ
1,5fb55fad26ed1e00351712f0,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,ref,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.097000Z,2020-11-18T17:53:49.097000Z,2020-11-19T13:44:31.752000Z,FAQ
2,5fb55fad26ed1e003c1712e5,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,submit,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.427000Z,2020-11-18T17:53:49.427000Z,2020-11-19T13:44:31.564000Z,FAQ
3,5fb55fad26ed1e003d1712f7,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,lammps,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.754000Z,2020-11-18T17:53:49.754000Z,2020-11-19T13:44:31.380000Z,FAQ
4,5fb55fae26ed1e003b1712e5,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,faq,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.159000Z,2020-11-18T17:53:50.159000Z,2020-11-19T13:44:31.193000Z,FAQ
5,5fb55fae26ed1e003a1712f3,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,graphs,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.488000Z,2020-11-18T17:53:50.488000Z,2020-11-19T13:44:31Z,FAQ
6,5fb55fae26ed1e00351712f4,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,manuscript,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.811000Z,2020-11-18T17:53:50.811000Z,2020-11-19T13:44:30.816000Z,FAQ


Pick the first record, and see its xml contents

In [5]:
record = records.iloc[0]
content = record.xml_content
print(content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>What are the potential formats?</question><answer><![CDATA[<b>Due to the range of formats, we are not able to guarantee that the interatomic potentials will work with particular software packages. Rather, we provide the interatomic potentials with appropriate citation information, and it is up to the user to determine whether a potential will work with given software and produce the expected results.</b></p>  <p>Steve Plimpton has a very nice description of various potential and file  formats on the <a href="http://lammps.sandia.gov/doc/pair_style.html" class="external">LAMMPS website</a>. These include the <b>Embedded-atom method (EAM) funcfl</b> and <b>setfl</b> formats, <b>Finnis-Sinclair</b> potentials, and <b>Modified embedded-atom method (MEAM)</b> potentials. More detailed information can be found there.   In addition to LAMMPS, the EAM funcfl and setfl formats are compatible with DYNAMO, the molecular dynami

## 3. Manage records

The methods associated with managing records requires having an account with the corresponding database and the correct privileges.

### 3.1. Upload a new record

New records can be uploaded to the database using the upload_record() method.

Parameters

- **template** (*str or pandas.Series*) The template or template title to associate with the record.
- **filename** (*str, optional*) Name of an XML file whose contents are to be uploaded.  Either filename or content required.
- **content** (*str or bytes, optional*) String content to upload. Either filename or content required.
- **title** (*str, optional*) Title to save the record as.  Optional if filename is given (title will be taken as filename without ext).
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being uploaded.
- **duplicatecheck** (*bool, optional*) If True (default), then a ValueError will be raised if a record already exists with the same template and title.  If False, no check is performed possibly allowing for multiple records with the same title to exist in the database.  Note: this check only searches the records that you have access to, so duplicates are possible from other users.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Use the content from the record queried above to upload a "test" record.

In [6]:
title = 'testrecord1'
curator.upload_record(template=template, title=title, content=content, verbose=True)

record testrecord1 (610171ef542fc6b84c74cbf7) successfully uploaded.


Alternatively, the names of any local XML files can be specified and the method will automatically read the contents.  The file name without path and extension will be used for the title if no title is given.

Also, note that the workspace is being set to 'Global Public Workspace'.

In [7]:
# Save content to local file
filename = 'testrecord2.xml'
with open(filename, 'w') as f:
    f.write(content)
    
# Upload from file
curator.upload_record(template=template, filename=filename, verbose=True,
                      workspace='Global Public Workspace')

# Delete local file (keep working directory clean)
Path(filename).unlink()

record testrecord2 (610171f5b392c35e3e74cb89) successfully uploaded.
record 610171f5b392c35e3e74cb89 assigned to workspace 5fb55e4826ed1e0015e846a9


### 3.2. Retrieve data records

The records that you own can be accessed using the get_records() and get_record() methods. get_records() will fetch all matching records, while get_record() will fetch a single record if exactly one match is found and will throw an error otherwise.

Parameters

- **template** (*str or pandas.Series, optional*) The template or template title to limit the search by.
- **title** (*str, optional*) The data record title to limit the search by.

In [8]:
curator.get_records(template=template)

,id,template,workspace,user_id,title,xml_content,creation_date,last_modification_date,last_change_date
0,610171f5b392c35e3e74cb89,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,testrecord2,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-07-28T15:04:21.573000Z,2021-07-28T15:04:21.573000Z,2021-07-28T15:04:23.033000Z
1,610171ef542fc6b84c74cbf7,5fb55fac26ed1e003b1712e1,None,2,testrecord1,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2021-07-28T15:04:15.161000Z,2021-07-28T15:04:15.161000Z,2021-07-28T15:04:15.161000Z
2,5fb55fae26ed1e00351712f4,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,manuscript,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.811000Z,2020-11-18T17:53:50.811000Z,2020-11-19T13:44:30.816000Z
3,5fb55fae26ed1e003a1712f3,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,graphs,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.488000Z,2020-11-18T17:53:50.488000Z,2020-11-19T13:44:31Z
4,5fb55fae26ed1e003b1712e5,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,faq,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:50.159000Z,2020-11-18T17:53:50.159000Z,2020-11-19T13:44:31.193000Z
5,5fb55fad26ed1e003d1712f7,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,lammps,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.754000Z,2020-11-18T17:53:49.754000Z,2020-11-19T13:44:31.380000Z
6,5fb55fad26ed1e003c1712e5,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,submit,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.427000Z,2020-11-18T17:53:49.427000Z,2020-11-19T13:44:31.564000Z
7,5fb55fad26ed1e00351712f0,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,ref,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:49.097000Z,2020-11-18T17:53:49.097000Z,2020-11-19T13:44:31.752000Z
8,5fb55fac26ed1e003a1712ef,5fb55fac26ed1e003b1712e1,5fb55e4826ed1e0015e846a9,2,formats,"<faq xmlns:xsi=""http://www.w3.org/2001/XMLSch...",2020-11-18T17:53:48.764000Z,2020-11-18T17:53:48.764000Z,2020-11-19T13:44:31.936000Z


In [9]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 610171f5b392c35e3e74cb89
template                                           5fb55fac26ed1e003b1712e1
workspace                                          5fb55e4826ed1e0015e846a9
user_id                                                                   2
title                                                           testrecord2
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
creation_date                                   2021-07-28T15:04:21.573000Z
last_modification_date                          2021-07-28T15:04:21.573000Z
last_change_date                                2021-07-28T15:04:23.033000Z
Name: 0, dtype: object


In [10]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>What are the potential formats?</question><answer><![CDATA[<b>Due to the range of formats, we are not able to guarantee that the interatomic potentials will work with particular software packages. Rather, we provide the interatomic potentials with appropriate citation information, and it is up to the user to determine whether a potential will work with given software and produce the expected results.</b></p>  <p>Steve Plimpton has a very nice description of various potential and file  formats on the <a href="http://lammps.sandia.gov/doc/pair_style.html" class="external">LAMMPS website</a>. These include the <b>Embedded-atom method (EAM) funcfl</b> and <b>setfl</b> formats, <b>Finnis-Sinclair</b> potentials, and <b>Modified embedded-atom method (MEAM)</b> potentials. More detailed information can be found there.   In addition to LAMMPS, the EAM funcfl and setfl formats are compatible with DYNAMO, the molecular dynami

### 3.3. Update an existing record

The content of a record in the database can be changed using the update_record() method.  Note that this only changes the record's content and all other metadata (user, workspace, database id,...) will remain unchanged.

- **record** (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the template and title parameters are ignored if given. Can contain the new conte
- **template** (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- **title** (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- **filename** (*str or Path, optional*) Path to file containing the new record content to upload. Either filename or content required.
- **content** (*str or bytes, optional*) New content to upload. Either filename or content required.
- **workspace** (*str or pandas.Series, optional*) If given, the record will be assigned to this workspace after successfully being updated.
- **verbose** (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Change the content of testrecord2 by replacing "download" with "find"

In [11]:
new_content = record.xml_content.replace('download', 'find')
print(new_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>What are the potential formats?</question><answer><![CDATA[<b>Due to the range of formats, we are not able to guarantee that the interatomic potentials will work with particular software packages. Rather, we provide the interatomic potentials with appropriate citation information, and it is up to the user to determine whether a potential will work with given software and produce the expected results.</b></p>  <p>Steve Plimpton has a very nice description of various potential and file  formats on the <a href="http://lammps.sandia.gov/doc/pair_style.html" class="external">LAMMPS website</a>. These include the <b>Embedded-atom method (EAM) funcfl</b> and <b>setfl</b> formats, <b>Finnis-Sinclair</b> potentials, and <b>Modified embedded-atom method (MEAM)</b> potentials. More detailed information can be found there.   In addition to LAMMPS, the EAM funcfl and setfl formats are compatible with DYNAMO, the molecular dynami

In [12]:
curator.update_record(record=record, content=new_content, verbose=True)

record testrecord2 (610171f5b392c35e3e74cb89) has been updated.


Retrieve the record again from the database showing nothing has changed except modification date and content

In [13]:
record = curator.get_record(template=template, title='testrecord2')
print(record)

id                                                 610171f5b392c35e3e74cb89
template                                           5fb55fac26ed1e003b1712e1
workspace                                          5fb55e4826ed1e0015e846a9
user_id                                                                   2
title                                                           testrecord2
xml_content               <faq  xmlns:xsi="http://www.w3.org/2001/XMLSch...
creation_date                                   2021-07-28T15:04:21.573000Z
last_modification_date                          2021-07-28T15:04:38.042000Z
last_change_date                                2021-07-28T15:04:38.047000Z
Name: 0, dtype: object


In [14]:
print(record.xml_content)

<faq  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" ><question>What are the potential formats?</question><answer><![CDATA[<b>Due to the range of formats, we are not able to guarantee that the interatomic potentials will work with particular software packages. Rather, we provide the interatomic potentials with appropriate citation information, and it is up to the user to determine whether a potential will work with given software and produce the expected results.</b></p>  <p>Steve Plimpton has a very nice description of various potential and file  formats on the <a href="http://lammps.sandia.gov/doc/pair_style.html" class="external">LAMMPS website</a>. These include the <b>Embedded-atom method (EAM) funcfl</b> and <b>setfl</b> formats, <b>Finnis-Sinclair</b> potentials, and <b>Modified embedded-atom method (MEAM)</b> potentials. More detailed information can be found there.   In addition to LAMMPS, the EAM funcfl and setfl formats are compatible with DYNAMO, the molecular dynami

### 3.4. Assign records to a workspace

The upload_record() and update_record() methods give the option to assign each record to a workspace if wanted.  After a record is added, the workspace can also be changed using the assign_records() method.

Parameters

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the records to.
- __records__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected records to assign to the workspace.  Cannot be given with ids, template, or title.
- __ids__ (*str or list, optional*) The ID(s) of the records to assign to the workspace.  Selecting records using ids has the least overhead. Cannot be given with records, template, or title.
- __template__ (*str or pandas.Series, optional*) The template or template title of records to assign to the workspace.  Cannot be given with records or ids.
- __title__ (*str, optional*) The title of a record to assign to the workspace. Cannot be given with records or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [15]:
curator.assign_records(workspace='Global Public Workspace', title='testrecord1', template=template, verbose=True)

record 610171ef542fc6b84c74cbf7 assigned to workspace 5fb55e4826ed1e0015e846a9


### 3.5. Delete a record

A record can also be deleted by the assigned user using the delete_record() method.

Parameters

- __record__ (*pandas.Series, optional*) A previously identified record to delete.  As this uniquely defines a record, the other parameters are ignored.
- __template__ (*str or pandas.Series, optional*) The template or template title associated with the record.  template + title values must uniquely identify one record.
- __title__ (*str, optional*) Title of the record to delete.  template + title values must uniquely identify one record.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [16]:
# Delete by identifying with title + template
curator.delete_record(title='testrecord1', template=template, verbose=True)

record testrecord1 (610171ef542fc6b84c74cbf7) has been deleted.


In [17]:
# Get a record first
record = curator.get_record(title='testrecord2', template=template)

# Delete by passing Series
curator.delete_record(record, verbose=True)

record testrecord2 (610171f5b392c35e3e74cb89) has been deleted.


## 4. Manage blobs (raw files)

The database can also store any non-record files as blob files that can later be retrieved and downloaded all at once.

Create blob file for testing

In [18]:
filename = 'test_blob.txt'

with open(filename, 'w') as f:
    f.write('This is my blob for testing')

### 4.1. Upload blob

Blobs can be added to the database using the upload_blob() method.  When a record is uploaded, it will be associated with the given filename (minus the path).

Parameters

- __filename__ (*str or Path*) The path/name of the file to upload.
- __blobbytes__ (*bytesIO, optional*) Pre-loaded file contents.  Allows for the contents of open file-like objects to be passed in and associated with the filename.  If not given, then the file filename will be read instead.
- __workspace__ (*str or pandas.Series, optional*) If given, the blob will be assigned to this workspace after successfully being uploaded.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

Returns

- __handle__ (*str*) The URL that can be used to retrieve the blob's contents.

In [19]:
handle = curator.upload_blob(filename=filename, verbose=True)
print(handle)

File "test_blob.txt" uploaded as blob "test_blob.txt" (61017222542fc6b84c74cbfc)
https://potentials.nist.gov/rest/blob/download/61017222542fc6b84c74cbfc/


### 4.2 Find blob metadata

The list of all blobs currently in the database, or all with an associated filename can be explored with the get_blobs() method.

Parameters

- __filename__ (*str, optional*) The name of the file to limit the search by.

A single blob can also be retrieved using the get_blob() method based on a filename, if it is unique in the database, or a database id.

Parameters

- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with filename
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with id.

__NOTE__: Exploring blob metadata appears to behave differently for some versions of CDCS, with the filename searches only showing blobs that the user owns, while id searches shows any blobs available to the user, either by being owned or in an available workspace.

- Identifying by filename only works for user blobs
- Identifying by id works for all blobs available to you (user + allowed workspaces)

In [20]:
curator.get_blobs()

,id,user_id,filename,handle,upload_date
0,606b4abf26ed1e002ad1fab1,2,bond_angle_scan.7z,https://potentials.nist.gov/rest/blob/download...,2021-04-05 17:37:03+00:00
1,606ee34ce5dbce002bc0f04c,2,W-H-H.7z,https://potentials.nist.gov/rest/blob/download...,2021-04-08 11:04:44+00:00
2,61017222542fc6b84c74cbfc,2,test_blob.txt,https://potentials.nist.gov/rest/blob/download...,2021-07-28 15:05:06+00:00


In [21]:
blobdata = curator.get_blob(filename=filename)
print(blobdata)

id                                      61017222542fc6b84c74cbfc
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
upload_date                            2021-07-28 15:05:06+00:00
Name: 0, dtype: object


In [22]:
curator.get_blob(id=blobdata.id)

id                                      61017222542fc6b84c74cbfc
user_id                                                        2
filename                                           test_blob.txt
handle         https://potentials.nist.gov/rest/blob/download...
upload_date                            2021-07-28 15:05:06+00:00
dtype: object

### 4.3 Retrieve blob contents

The contents of a blob can then be retrieved using the get_blob_contents() method.

Paramters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.

Returns

- __content__ (*bytes*) The blob's bytes contents.

In [23]:
print(curator.get_blob_contents(filename=filename))

b'This is my blob for testing'


Alternatively, the blob can be saved directly to a local file based on its associated filename with the download_blob() method.

Parameters

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __savedir__ (*str or Path, optional*) The directory to save the file to.  Default value uses the current working directory.

In [24]:
curator.download_blob(filename=filename)

### 4.4. Assign blobs to a workspace

Just like records, blobs can also be assigned to workspaces.  This can be done during uploading, or it can be done after uploading using the assign_blobs() method.

- __workspace__ (*str or pandas.Series*) The workspace or workspace title to assign the blobs to.
- __blobs__ (*pandas.Series or pandas.DataFrame, optional*) Pre-selected blobs to assign to the workspace.  Cannot be given with ids or filename.
- __ids__ (*str or list, optional*) The ID(s) of the blobs to assign to the workspace.  Selecting blobs using ids has the least overhead. Cannot be given with blobs or filename.
- __filename__ (*str, optional*) The name of the blob file to assign to the workspace.  Cannot be given with blobs or ids.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [25]:
curator.assign_blobs(workspace='Global Public Workspace', filename=filename, verbose=True)

blob 61017222542fc6b84c74cbfc assigned to workspace 5fb55e4826ed1e0015e846a9


### 4.5. Delete blob

Finally, blobs can also be deleted using the delete_blob() method.

- __blob__ (*pandas.Series, optional*) The blob metadata for a blob, as recieved from get_blobs() or get_blob().  Cannot be combined with id or filename.
- __id__ (*str, optional*) The unique ID associated with the blob. Cannot be combined with blob or filename.
- __filename__ (*str, optional*) The name of the file to limit the search by, which must be unique.  Cannot be combined with blob or id.
- __verbose__ (*bool, optional*) Setting this to True will print extra status messages.  Default value is False.

In [26]:
curator.delete_blob(filename=filename, verbose=True)

Successfully deleted blob "test_blob.txt" (61017222542fc6b84c74cbfc)


In [34]:
Path(filename).unlink()

## 5. Supporting methods

### 5.1. Finding workspace metadata

Metadata for the available workspaces can be explored using the get_workspaces() and get_workspace() methods.

Parameters

title: (str, optional) The workspace title to limit the search by.

In [27]:
curator.get_workspaces()

,id,title,owner,is_public
0,5fb55e4826ed1e0015e846a9,Global Public Workspace,None,True
1,606b0e5b26ed1e003a28f025,Three body scan,2,False


In [28]:
curator.get_workspace(title='Global Public Workspace')

id           5fb55e4826ed1e0015e846a9
title         Global Public Workspace
owner                            None
is_public                        True
Name: 0, dtype: object

The attribute global_workspace also retrieves the Global Public Workspace 

In [29]:
workspace = curator.global_workspace
print(workspace)

id           5fb55e4826ed1e0015e846a9
title         Global Public Workspace
owner                            None
is_public                        True
Name: 0, dtype: object


### 5.2. Finding template metadata

A list of all available templates can be retrieved using the template_titles attribute.

In [30]:
curator.template_titles

['Potential',
 'Family',
 'FAQ',
 'Action',
 'PotentialProperties',
 'Request',
 'free_surface',
 'stacking_fault',
 'point_defect',
 'crystal_prototype',
 'potential_LAMMPS',
 'Citation',
 'relaxed_crystal',
 'calculation_diatom_scan',
 'calculation_isolated_atom',
 'potential_LAMMPS_KIM',
 'calculation_bond_angle_scan']

Metadata for the available templates can be explored using the get_templates() and get_template() methods.

Parameters

- __title__ (*str, optional*): The template title to limit the search by.
- __is_disabled__ (*bool, optional*): If True, then disabled templates will be returned.  If False (default), then active templates will be returned.
- __current__ (bool, optional*): If True (default), only current template versions will be returned.
- __useronly__ (*bool, optional*): If True, only a user's templates are returned. If False (default), then all global templates are returned.

In [31]:
curator.get_templates()

,id,user,filename,content,hash,dependencies,title
0,5fb7035126ed1e0037179acc,None,Potential4.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",c35b164c2947bdab2c6dd8566f84584a4a3eef7f,[],Potential
1,5fb55fab26ed1e003c1712e1,None,record-interatomic-potential-family.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",493be99cbc23614f898354553ea0d307009673d3,[],Family
2,5fb55fac26ed1e003b1712e1,None,record-interatomic-potential-faq.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",2dc0be555da041f782a7435c377ea7a20f28d3cd,[],FAQ
3,5fb55faf26ed1e003d1712f9,None,record-interatomic-potential-action.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",f1c3de8a64553d81bd10cfc0cb24dd3cc129c710,[],Action
4,5fb55ff126ed1e003c17137f,None,record-per-potential-properties.xsd,"<?xml version=""1.0"" encoding=""UTF-8"" standalon...",ea7eeed57e0cea9b15dbda4b32c08266201a7e5a,[],PotentialProperties
5,5fb5614a26ed1e00351714c9,None,Request3,"<xsd:schema xmlns:xsd=""http://www.w3.org/2001/...",d108bacceb44a2789544ade71018b7f5ceacca1b,[],Request
6,5fb5617026ed1e003517151b,None,record-free-surface.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",8d2ff690d7e5558b851769f75bad8d90af47102d,[],free_surface
7,5fb561a226ed1e003b17159d,None,record-stacking-fault.xsd,"<?xml version=""1.0""?>\n<xsd:schema xmlns:xsd=""...",aaa696ec7723f68e5be8280c8cafab69e08cf3f3,[],stacking_fault
8,5fb561a726ed1e0038171153,None,record-point-defect.xsd,"<?xml version=""1.0""?>\n<xsd:schema xmlns:xsd=""...",4c8bc7e9d8369cdea5c61b27b418c84d3a350a2d,[],point_defect
9,60f06f91e3f5c92595740437,None,crystal_prototype.xsd,"<?xml version=""1.0""?>\r\n<xsd:schema xmlns:xsd...",c270b1e6186a7d0a34489596781e02af1b80d006,[],crystal_prototype


In [32]:
curator.get_template('Citation')

id                                       5fb562c626ed1e002a1713ef
user                                                         None
filename                record-interatomic-potential-citation.xsd
content         <?xml version="1.0" encoding="UTF-8" standalon...
hash                     893099854c8ac392bad133ce71ac34f60407ab6f
dependencies                                                   []
title                                                    Citation
Name: 0, dtype: object

The get_template_managers() method fetches metadata associated with the various versions of templates in the database. 

Parameters

- __title__ (*str, optional*) The template title to limit the search by.
- __is_disabled__ (*bool, optional*) If True, then disabled templates will be returned.  If False (default), then active templates will bereturned.
- __useronly__ (*bool, optional*) If True, only a user's templates are returned. If False (default), then all global templates arereturned.

In [33]:
curator.get_template_managers('Citation')

,id,title,user,versions,current,is_disabled,disabled_versions,_cls
0,5fb562c626ed1e002a1713f0,Citation,None,[5fb562c626ed1e002a1713ef],5fb562c626ed1e002a1713ef,False,[],VersionManager.TemplateVersionManager
